In [23]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
import chromadb
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers



In [25]:
from langchain_pinecone.embeddings import PineconeEmbeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
from pinecone import Pinecone

In [11]:
# !pip install pypdf

In [12]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob = "*.pdf",
                             loader_cls = PyPDFLoader)
    documents = loader.load()
    
    return documents

In [13]:
extracted_text = load_pdf("D:\Medical-Chat-Bot-using-Llama\data")

In [9]:
#extracted_text

In [6]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="jlallas/Meta-Llama-3-8B-Instruct-Q4_K_M-GGUF",
    filename="meta-llama-3-8b-instruct-q4_k_m.gguf",
    cache_dir="./models"
)
print("✅ Downloaded to:", model_path)


✅ Downloaded to: ./models\models--jlallas--Meta-Llama-3-8B-Instruct-Q4_K_M-GGUF\snapshots\e05e01f6dfd1aff3c39778d049aa351b233c7439\meta-llama-3-8b-instruct-q4_k_m.gguf


In [49]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_huggingface_embedings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [50]:
embeddings = download_huggingface_embedings()


C:\Users\Dell\AppData\Local\Temp\ipykernel_2624\1412518668.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DEC165E0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DCD43A90>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2DD7A05E0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2D99A9730>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E2D99A9460>
Unclosed client session
client_session: <a

In [51]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
# result = embeddings.embed_query("Hello, world! How are you?")
# print(result)


In [ ]:
def text_splitter(extracted_text):
    text_split = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
    return text_split.split_documents(extracted_data)


In [15]:
text_chunks = text_splitter(extracted_text)

In [ ]:
len(text_chunks)

4251

In [ ]:
# text_chunks
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# print(PINECONE_API_KEY)

In [78]:

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-bot")


In [71]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [72]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

In [73]:
vector_store.add_documents(documents=text_chunks, ids=uuids)

['5953038d-635a-4711-a118-2ec96e005017',
 '5537352b-3c66-4229-8a54-ab7de5373245',
 '7b9010da-8bec-4502-b66e-5e83380d619b',
 '601507d8-d7f9-41d0-888f-7e72c09682c7',
 '453717dc-3465-4284-99d8-e6e7d2e73ffb',
 '54b1ceaf-97d1-45b8-a7d0-d6d16304cd7f',
 '6c649137-524b-4d41-86f4-06cda66b7e4d',
 'bfe18c00-67ad-4794-926b-6be0a45476fe',
 'd5009173-456f-405e-96e1-e99fbc737a65',
 '7b1666c8-4164-4865-922f-857e24d12977',
 '09744b61-0cd8-4b8c-a678-5174a70a8246',
 'c52f220b-07af-4dcc-b46b-74d089c780e5',
 '57410986-118a-4045-8be6-669254ee38d1',
 '7103666d-60e2-47fd-9cf2-297cdd8ef1e1',
 'a0b91a31-f574-446f-87d0-9848840e671a',
 '4382181b-0766-4af9-9f34-6167ec07c6c4',
 '9cc84d2f-e11d-458d-88c0-a8c00cbac7cb',
 'c6c78dd5-9695-469f-8467-0fb32f5ebb69',
 '05c9034c-2fe6-43c5-bbd3-2ce22a19fdca',
 '9c2cc6cf-4b98-4e4b-b191-f96cb265dcab',
 '570554cc-a492-4376-b293-c09b2ec2f597',
 '40adf1a3-791e-461f-be97-77290458b850',
 '9d3f4087-3a9d-4024-bf98-a721a034d5be',
 '86635210-97a4-439b-bf32-bc8128bc0d00',
 '2d0d5bcd-d905-